<a href="https://colab.research.google.com/github/QuinnStraus/CSCI-1470-final/blob/main/DL_Final_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pypianoroll

     |████████████████████████████████| 5.6 MB 38.2 MB/s 
     |████████████████████████████████| 51 kB 8.3 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=9022aaef2f5723b07831c0fa9ae1ac53dd60bc6dd005b0c519266532dfa741d7
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [4]:
import pypianoroll
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import json
from music21 import *
import pdb


In [5]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [6]:
!ls 'gdrive/Shared drives/CSCI1470 Final Project'
root_dir = 'gdrive/Shared drives/CSCI1470 Final Project'
data_dir = 'gdrive/Shared drives/CSCI1470 Final Project/lpd_5/lpd_5_cleansed'

cleansed_ids.txt  lpd_5  midi_info.json  msd_tagtraum_cd1.cls  songs.txt


In [ ]:
#def generate_path(msd_id, tag):
#    return os.path.join(root_dir, 'lpd_5', 'lpd_5_cleansed', msd_id[2], msd_id[3], msd_id[4], msd_id, tag + '.npz')

In [7]:
cleansed_ids = pd.read_csv(os.path.join(root_dir, 'cleansed_ids.txt'), delimiter = '    ')
msd_lpd = {a:b for a, b in zip(cleansed_ids.iloc[:, 1], cleansed_ids.iloc[:, 0])}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [8]:
genre_file_dir = os.path.join(root_dir, 'msd_tagtraum_cd1.cls')
ids = []
genres = []
with open(genre_file_dir) as f:
    line = f.readline()
    while line:
        if line[0] != '#':
            split = line.strip().split("\t")
            if len(split) == 2:
                ids.append(split[0])
                genres.append(split[1])
            elif len(split) == 3:
                ids.append(split[0]) 
                ids.append(split[0])
                genres.append(split[1])
                genres.append(split[2])
        line = f.readline()
genre_df = pd.DataFrame(data={"TrackID": ids, "Genre": genres})

genre_dict = genre_df.groupby('TrackID')['Genre'].apply(lambda x: x.tolist()).to_dict()

In [ ]:
print(len(genre_dict))
print(genre_df)
print(len(msd_lpd))
print(len(cleansed_ids))

133676
                   TrackID     Genre
0       TRAAAAK128F9318786  Pop_Rock
1       TRAAAAW128F429D538       Rap
2       TRAAABD128F429CF47  Pop_Rock
3       TRAAAED128E0783FAB      Jazz
4       TRAAAED128E0783FAB     Vocal
...                    ...       ...
169198  TRZZZMY128F426D7A2    Reggae
169199  TRZZZMY128F426D7A2  Pop_Rock
169200  TRZZZYR128F92F0796  Pop_Rock
169201  TRZZZYV128F92E996D       RnB
169202  TRZZZYV128F92E996D   New Age

[169203 rows x 2 columns]
21424
21424


In [9]:
# copied the path functions from the article
# Utility functions for retrieving paths
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)

# Load the midi npz file from the LMD cleansed folder
def get_midi_npz_path(msd_id, midi_md5):
    return os.path.join(data_dir,
                        msd_id_to_dirs(msd_id), midi_md5 + '.npz')
    
# Load the midi file from the Music Dataset folder
#def get_midi_path(msd_id, midi_md5):
#    return os.path.join(music_dataset_lpd_dir,
#                        msd_id_to_dirs(msd_id), midi_md5 + '.mid')
    
# Open the cleansed ids - cleansed file ids : msd ids
cleansed_ids = pd.read_csv(os.path.join(root_dir, 'cleansed_ids.txt'), delimiter = '    ', header = None)
lpd_to_msd_ids = {a:b for a, b in zip(cleansed_ids[0], cleansed_ids[1])}
msd_to_lpd_ids = {a:b for a, b in zip(cleansed_ids[1], cleansed_ids[0])}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [ ]:
#with open(os.path.join(root_dir, 'midi_info.json'), 'r') as outfile:
#  lmd_metadata = json.load(outfile)

In [10]:
# converts npz files to midi files, no need to run again as all midi files already saved
for msd_file_name in cleansed_ids[1]:
  lpd_file_name = msd_to_lpd_ids[msd_file_name] 
  # msd_file_name ex: TRLOCTJ12903CDEC24, lpd_file_name: 6b16a2b1720d11263069a9a189785dfb
  npz_path = get_midi_npz_path(msd_file_name, lpd_file_name)
  multitrack = pypianoroll.load(npz_path)
  pm = pypianoroll.to_pretty_midi(multitrack)
  new_midi_path = npz_path[:-4] + '.mid'
  pypianoroll.write(new_midi_path, multitrack)

KeyboardInterrupt: ignored

In [46]:
#added newline after stop, took max 200 tokens

# save songs to text file after tokenizing the midi files
allSongs = []
stopInd = 10
song_ind=0
f = open(os.path.join(root_dir, "songs.txt"), "w")
for msd_file_name in cleansed_ids[1]:
  elt_ind = 0
  lpd_file_name = msd_to_lpd_ids[msd_file_name]
  npz_path = get_midi_npz_path(msd_file_name, lpd_file_name)
  new_midi_path = npz_path[:-4] + '.mid'
  print("loaded data")
  midi = converter.parse(new_midi_path)
  #print("opened file")
  #open midi file and select piano part
  for part in instrument.partitionByInstrument(midi):
    if part.getInstrument().instrumentName == 'Piano':
      curSong = []
      #add notes, chords, and rests to list of strings
      for i in range(min(len(part), 200)):
        elt = part[i]
        if isinstance(elt, note.Note):
          toWrite = str(elt.pitch) + 'd' + str(elt.duration)[27:-1] + ' br '
          curSong.append(toWrite)
          f.write(toWrite)
        elif isinstance(elt, chord.Chord):
          dur = str(elt.duration)[27:-1]
          if dur == '0.0' and isinstance(part[elt_ind+1], note.Rest):
              dur = str(part[elt_ind+1].duration)[27:-1]
          toWrite = ''
          for nt in elt.pitches:
            toWrite += str(nt) + 'd' + dur + ' '
          toWrite += ' br '
          f.write(toWrite)
        elt_ind += 1
  song_ind+=1
  f.write('<stop>\n')
  if song_ind % stopInd == 0:
    print(song_ind)
    f.close()
    break

loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
10


In [45]:
#Test function using pypianoroll, loads faster at the loss of some information

# save songs to text file after tokenizing the midi files
allSongs = []
stopInd = 20
song_ind=0
f = open(os.path.join(root_dir, "songs.txt"), "w")
for msd_file_name in cleansed_ids[1]:
  elt_ind = 0
  lpd_file_name = msd_to_lpd_ids[msd_file_name]
  npz_path = get_midi_npz_path(msd_file_name, lpd_file_name)
  new_midi_path = npz_path[:-4] + '.npz'
  print("loaded data")
  midi = pypianoroll.load(new_midi_path)
  #open midi file and select piano part
  for part in midi.tracks:
    if part.name == 'Piano':
      roll = part.pianoroll
      if roll != np.zeros([0, 128]):
        #add notes, chords, and rests to list of strings
        for i in range(min(part.pianoroll.shape[0], 200)):
          toWrite = ''
          elts = np.nonzero(part[i])
          for nt in elts:
            toWrite += str(nt) + ' '
          toWrite += ' br '
          f.write(toWrite)
          elt_ind += 1
  song_ind+=1
  f.write('<stop>\n')
  if song_ind % stopInd == 0:
    print(song_ind)
    f.close()
    break

loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
loaded data
20


In [ ]:

for nt in elts:
  toWrite += str(nt) + ' '

In [ ]:
# convert tokenized text file back into midi
def text_to_song(file_path):
  """ converts text file with songs broken up by <stop> and br delimiters into a 
  list of elements for each song"""
  with open(file_path) as f:
    text = f.read()
  songs = [x.split(' br ') for x in text.split('<stop>')]
  return songs

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for elt in prediction_output:
        # pattern is a br
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    return midi_stream

In [ ]:
text_to_song(root_dir + '/songs.txt')

[['F1d0.5 F2d0.5 ',
  'G#4d1.0 C5d1.0 F5d1.0 ',
  'F2d0.5 F3d0.5 ',
  'F2d0.5 F3d0.5 ',
  'F2d0.5 F3d0.5 ',
  'E2d0.5 E3d0.5 ',
  'C5d1.0 G5d1.0 C6d1.0 ',
  'C3d0.5 C4d0.5 ',
  'C3d0.5 C4d0.5 ',
  'C3d0.5',
  'C4d1.5',
  'G#2d0.5 G#3d0.5 ',
  'F4d1.0 G#4d1.0 ',
  'F2d0.5 F3d0.5 ',
  'F2d0.5 F3d0.5 ',
  'F2d0.5 F3d0.5 ',
  'E2d0.5 E3d0.5 ',
  'C4d1.0 E4d1.0 G4d1.0 ',
  'C2d0.5 C3d0.5 ',
  'C2d0.5 C3d0.5 ',
  'C2d0.5 C3d0.5 ',
  'G#1d0.5 G#2d0.5 ',
  'G#2d0.5 G#3d0.5 C4d0.5 ',
  'G2d0.5 G3d0.5 B-3d0.5 ',
  'F2d0.5 F3d0.5 G#3d0.5 ',
  'E2d0.5 E3d0.5 ',
  'G3d1.0',
  'C2d0.5 C3d0.5 ',
  'C2d0.5 C3d0.5 ',
  'B-1d0.5 B-2d0.5 ',
  'G#1d0.5 G#2d0.5 F4d0.5 C5d0.5 ',
  'G#1d0.5 G#2d0.5 F4d0.5 C5d0.5 F5d0.5 ',
  'G1d0.5 G2d0.5 B-4d0.5 ',
  'E5d0.75',
  'F1d0.5 F2d0.5 C4d0.5 G#4d0.5 ',
  'F5d0.25',
  'C2d0.5 C3d0.5 ',
  'G4d1.0 C5d1.0 E5d1.0 G5d1.0 ',
  'C3d0.5 C4d0.5 ',
  'C3d0.5 C4d0.5 ',
  'B-2d0.5 B-3d0.5 ',
  'G#2d0.5 G#3d0.5 ',
  'G#2d0.5 G#3d0.5 ',
  'G2d0.5 G3d0.5 ',
  'F2d0.5 F3d0.5 ',
  